### --- Day 12: Christmas Tree Farm ---
You're almost out of time, but there can't be much left to decorate. Although there are no stairs, elevators, escalators, tunnels, chutes, teleporters, firepoles, or conduits here that would take you deeper into the North Pole base, there is a ventilation duct. You jump in.

After bumping around for a few minutes, you emerge into a large, well-lit cavern full of Christmas trees!

There are a few Elves here frantically decorating before the deadline. They think they'll be able to finish most of the work, but the one thing they're worried about is the presents for all the young Elves that live here at the North Pole. It's an ancient tradition to put the presents under the trees, but the Elves are worried they won't fit.

The presents come in a few standard but very weird shapes. The shapes and the regions into which they need to fit are all measured in standard units. To be aesthetically pleasing, the presents need to be placed into the regions in a way that follows a standardized two-dimensional unit grid; you also can't stack presents.

As always, the Elves have a summary of the situation (your puzzle input) for you. First, it contains a list of the presents' shapes. Second, it contains the size of the region under each tree and a list of the number of presents of each shape that need to fit into that region. For example:
```
0:
###
##.
##.

1:
###
##.
.##

2:
.##
###
##.

3:
##.
###
##.

4:
###
#..
###

5:
###
.#.
###
```
```
4x4: 0 0 0 0 2 0
12x5: 1 0 1 0 2 2
12x5: 1 0 1 0 3 2
```
The first section lists the standard present shapes. For convenience, each shape starts with its index and a colon; then, the shape is displayed visually, where # is part of the shape and . is not.

The second section lists the regions under the trees. Each line starts with the width and length of the region; 12x5 means the region is 12 units wide and 5 units long. The rest of the line describes the presents that need to fit into that region by listing the quantity of each shape of present; 1 0 1 0 3 2 means you need to fit one present with shape index 0, no presents with shape index 1, one present with shape index 2, no presents with shape index 3, three presents with shape index 4, and two presents with shape index 5.

Presents can be rotated and flipped as necessary to make them fit in the available space, but they have to always be placed perfectly on the grid. Shapes can't overlap (that is, the # part from two different presents can't go in the same place on the grid), but they can fit together (that is, the . part in a present's shape's diagram does not block another present from occupying that space on the grid).

The Elves need to know how many of the regions can fit the presents listed. In the above example, there are six unique present shapes and three regions that need checking.

The first region is 4x4:
```
....
....
....
....
```
In it, you need to determine whether you could fit two presents that have shape index 4:
```
###
#..
###
```
After some experimentation, it turns out that you can fit both presents in this region. Here is one way to do it, using A to represent one present and B to represent the other:
```
AAA.
ABAB
ABAB
.BBB
```
The second region, 12x5: 1 0 1 0 2 2, is 12 units wide and 5 units long. In that region, you need to try to fit one present with shape index 0, one present with shape index 2, two presents with shape index 4, and two presents with shape index 5.

It turns out that these presents can all fit in this region. Here is one way to do it, again using different capital letters to represent all the required presents:
```
....AAAFFE.E
.BBBAAFFFEEE
DDDBAAFFCECE
DBBB....CCC.
DDD.....C.C.
```
The third region, 12x5: 1 0 1 0 3 2, is the same size as the previous region; the only difference is that this region needs to fit one additional present with shape index 4. Unfortunately, no matter how hard you try, there is no way to fit all of the presents into this region.

So, in this example, 2 regions can fit all of their listed presents.

Consider the regions beneath each tree and the presents the Elves would like to fit into each of them. How many of the regions can fit all of the presents listed?

In [49]:
with open('data.txt', 'r') as f:
        content = f.read().strip()
    
parts = content.split('\n\n')
shape_map = {}
tasks = []

lines = content.splitlines()
buffer = []
current_id = None

for line in lines:
    line = line.strip()
    if not line: continue
    
    if ':' in line and 'x' not in line:
        if current_id is not None: shape_map[current_id] = buffer
        current_id = int(line.replace(':', ''))
        buffer = []
    elif 'x' in line and ':' in line:
        if current_id is not None and buffer:
            shape_map[current_id] = buffer
            current_id = None
            buffer = []
        
        dim_part, counts_part = line.split(':')
        w, h = map(int, dim_part.split('x'))
        counts = list(map(int, counts_part.strip().split()))
        
        presents_to_fit = []
        for p_id, count in enumerate(counts):
            presents_to_fit.extend([p_id] * count)
    
        tasks.append({'w': w, 'h': h, 'presents': presents_to_fit})
    else:
        buffer.append(line)

if current_id is not None and buffer: shape_map[current_id] = buffer

In [72]:
def get_coords(shape_lines):
    coords = set()
    for r, row in enumerate(shape_lines):
        for c, char in enumerate(row):
            if char == '#': coords.add((r, c))
    return coords

In [73]:
def normalize(coords):
    if not coords: return frozenset()
    min_r = min(r for r, c in coords)
    min_c = min(c for r, c in coords)
    return frozenset((r - min_r, c - min_c) for r, c in coords)

In [94]:
def generate_base_variations(shape_lines):
    base = get_coords(shape_lines)
    variations = set()
    current = base
    for _ in range(2): 
        for _ in range(4):
            variations.add(normalize(current))
            current = set((c, -r) for r, c in current)
        current = set((r, -c) for r, c in base)
        base = current
    return list(variations)

In [95]:
def coords_to_bitmask(coords, offset_r, offset_c, w, h):
    mask = 0
    for r, c in coords:
        nr, nc = r + offset_r, c + offset_c
        if not (0 <= nr < h and 0 <= nc < w):
            return None
        bit_pos = nr * w + nc
        mask |= (1 << bit_pos)
    return mask

In [103]:
def solve_region_fast(w, h, presents_ids, shape_data):
    presents_info = []
    for pid in presents_ids:
        size = len(shape_data[pid][0])
        presents_info.append((size, pid))
    
    presents_info.sort(key=lambda x: x[0], reverse=True)
    sorted_pids = [p[1] for p in presents_info]

    possible_moves = {} 
    
    unique_pids = set(sorted_pids)
    for pid in unique_pids:
        moves = []
        variations = shape_data[pid]
        for var_coords in variations:
            max_r = max(r for r, c in var_coords)
            max_c = max(c for r, c in var_coords)
            
            for r in range(h - max_r):
                for c in range(w - max_c):
                    mask = coords_to_bitmask(var_coords, r, c, w, h)
                    if mask is not None:
                        moves.append(mask)
        possible_moves[pid] = sorted(list(set(moves)), reverse=True)

    def backtrack(idx, board_mask):
        if idx == len(sorted_pids):
            return True
        
        pid = sorted_pids[idx]
        
        for move_mask in possible_moves[pid]:
            if (board_mask & move_mask) == 0:
                if backtrack(idx + 1, board_mask | move_mask):
                    return True
        return False

    return backtrack(0, 0)

In [104]:
shape_data_coords = {}
for pid, lines in shape_map.items():
    shape_data_coords[pid] = generate_base_variations(lines)

success_count = 0
for i, task in enumerate(tasks):
    total_present_area = 0
    for pid in task['presents']:
        total_present_area += len(shape_data_coords[pid][0])
    if total_present_area > task['w'] * task['h']: continue
    if solve_region_fast(task['w'], task['h'], task['presents'], shape_data_coords): success_count += 1
        
success_count

492

That's the right answer! You are one gold star closer to decorating the North Pole.